In [ ]:
import os
import logging
from typing import Any, Set


from automata.config.config_base import AgentConfigName
from automata.singletons.dependency_factory import dependency_factory, DependencyFactory
from automata.singletons.py_module_loader import py_module_loader
from automata.symbol.graph import SymbolGraph
from automata.symbol_embedding import SymbolCodeEmbedding
from automata.symbol_embedding.vector_databases import (
    ChromaSymbolEmbeddingVectorDatabase,
)

    
logger = logging.getLogger(__name__)

In [ ]:
py_module_loader.reset()
dependency_factory.reset()
project_name = "automata"
py_module_loader.initialize(f"..", project_name) # ! Can replace with your own project


In [ ]:
symbol_graph = SymbolGraph(
    os.path.join(DependencyFactory.DEFAULT_SCIP_FPATH, f"{project_name}.scip")
)

code_embedding_db = ChromaSymbolEmbeddingVectorDatabase(
    project_name,
    persist_directory=DependencyFactory.DEFAULT_CODE_EMBEDDING_FPATH,
    factory=SymbolCodeEmbedding.from_args,
)

In [ ]:
symbol_graph.is_synchronized = True
len(symbol_graph.get_sorted_supported_symbols()),len(code_embedding_db)

In [ ]:
symbol_graph.get_sorted_supported_symbols()[-1]

In [ ]:
code_embedding_db.get_all_ordered_embeddings()[-1].symbol

In [ ]:
dependency_factory.set_overrides(
    **{
        "symbol_graph": symbol_graph,
        "code_embedding_db": code_embedding_db,
    }
)


symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')


In [ ]:
raw_ranks = symbol_search.symbol_rank.get_ordered_ranks()
print(f"Demonstrating SymbolRank (on {project_name}) \n")
for i,rank in enumerate(raw_ranks[0:10]):
    print(f"rank {i} = {rank[0].full_dotpath} with rank {rank[1]:.3f}")

In [ ]:
query = "How do I create an agent?"

embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')
code_embeddings = symbol_code_embedding_handler.get_all_ordered_embeddings()

print(f"Demonstrating Similarity Search (on {project_name}) for query = {query} \n")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[0:10]):
    print(f"rank {i} = {rank[0].full_dotpath} with distance {rank[1]:.3f}")

In [ ]:
symbol_rank_search_results = symbol_search.get_symbol_rank_results(query)
print(f"Demonstrating SymbolRank (on {project_name}) for query = {query} \n")
for i,rank in enumerate(symbol_rank_search_results[0:10]):
    print(f"rank {i} = {rank[0].full_dotpath} with rank {rank[1]:.3f}")